In [1]:
import os 
from utils.RagPipeline import Ragpipeline
from utils.config import config

In [2]:
selected_model= "gpt-4o-mini"
root_dir = ".cache/files"
selected_paper = 'example'

In [3]:
config["llm_predictor"]["model_name"] = selected_model
db_path = os.path.join(root_dir, selected_paper + '/db').replace('\\','/')
db_path

'.cache/files/example/db'

In [4]:

chain = Ragpipeline(db_path, config)

In [5]:
ensemble_retriever = chain.init_ensemble_retriever()

In [6]:
query = "Abstract을 설명해주세요"
ensemble_result = ensemble_retriever.invoke(query)

ensemble_result

[Document(metadata={'category': 'heading1', 'section': 'Abstract', 'page': 0, 'id': 2, 'section_summary': '- Research focus: dense video captioning for untrimmed video events.\n- Methodology: multitasking approach for event localization and captioning.\n- Challenge: using only visual input due to lack of semantic content.\n- Proposed framework: inspired by human cognitive information processing.\n- Key feature: external memory for incorporating prior knowledge.\n- Memory retrieval: cross-modal video-to-text matching.\n- Design: versatile encoder and decoder with visual and textual cross-attention modules.\n- Datasets used: ActivityNet Captions and YouCook2.\n- Results: promising performance without extensive pretraining from large video datasets.\n- Code availability: https://github.com/ailab-kyunghee/CM2_DVC.'}, page_content='Abstract'),
 Document(metadata={'category': 'heading1', 'section': '1. Introduction', 'page': 0, 'id': 4, 'section_summary': '- The field of video captioning is 

In [7]:
query = "Feature Aggregation에 대해서 더 자세히 알려줘."
ensemble_result = ensemble_retriever.invoke(query)

ensemble_result

[Document(metadata={'category': 'heading2', 'section': '3. Method', 'page': 2, 'id': 34, 'section_summary': '- **Goal**: Improve event-level localization and captioning from untrimmed video using prior knowledge.\n- **Framework**: Introduced CM2, designed for cross-modal memory retrieval.\n- **Key Components**:\n  - **Memory Construction**: External memory bank created using sentence-level features from training data (e.g., ActivityNet Caption).\n  - **Segment-level Retrieval**: Input video divided into W temporal anchors; segment-level visual features extracted using CLIP ViT-L/14.\n  - **Feature Aggregation**: Average pooling of K retrieved sentence features for each anchor.\n- **Encoder-Decoder Architecture**: Utilizes a versatile structure incorporating both visual and text features for localization and captioning.\n- **Parallel Heads**: \n  - **Localization Head**: Predicts start time, end time, and confidence for event segments.\n  - **Captioning Head**: Uses deformable soft atte

In [8]:
ensemble_result[0].page_content

'3.2. Versatile Encoder-Decoder'

In [9]:
ensemble_result[0].metadata

{'category': 'heading2',
 'section': '3. Method',
 'page': 2,
 'id': 34,
 'section_summary': '- **Goal**: Improve event-level localization and captioning from untrimmed video using prior knowledge.\n- **Framework**: Introduced CM2, designed for cross-modal memory retrieval.\n- **Key Components**:\n  - **Memory Construction**: External memory bank created using sentence-level features from training data (e.g., ActivityNet Caption).\n  - **Segment-level Retrieval**: Input video divided into W temporal anchors; segment-level visual features extracted using CLIP ViT-L/14.\n  - **Feature Aggregation**: Average pooling of K retrieved sentence features for each anchor.\n- **Encoder-Decoder Architecture**: Utilizes a versatile structure incorporating both visual and text features for localization and captioning.\n- **Parallel Heads**: \n  - **Localization Head**: Predicts start time, end time, and confidence for event segments.\n  - **Captioning Head**: Uses deformable soft attention LSTM for 

In [10]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""

    context = ""
    
    for doc in docs:
        metadata = doc.metadata
        section = metadata['section']
        section_summary = metadata['section_summary']
        metadata_context = f"#Section : {section} \n #Section_summary: {section_summary} \n #Content: {doc.page_content}"
        context += metadata_context
        context += '\n\n'
    return context

In [11]:
context = format_docs(ensemble_result)

In [12]:
print(context)

#Section : 3. Method 
 #Section_summary: - **Goal**: Improve event-level localization and captioning from untrimmed video using prior knowledge.
- **Framework**: Introduced CM2, designed for cross-modal memory retrieval.
- **Key Components**:
  - **Memory Construction**: External memory bank created using sentence-level features from training data (e.g., ActivityNet Caption).
  - **Segment-level Retrieval**: Input video divided into W temporal anchors; segment-level visual features extracted using CLIP ViT-L/14.
  - **Feature Aggregation**: Average pooling of K retrieved sentence features for each anchor.
- **Encoder-Decoder Architecture**: Utilizes a versatile structure incorporating both visual and text features for localization and captioning.
- **Parallel Heads**: 
  - **Localization Head**: Predicts start time, end time, and confidence for event segments.
  - **Captioning Head**: Uses deformable soft attention LSTM for generating captions.
  - **Event Counter**: Predicts the numbe

In [13]:
result = chain.answer_generation(query)

print(result)

'해당 논문의 Versatile Encoder-Decoder는 비디오와 텍스트 모달리티를 효과적으로 처리하기 위해 설계된 구조입니다. 이 구조는 두 가지 주요 기능을 수행하는데, 첫째는 이벤트 쿼리(event queries)를 통해 시간적 및 의미적으로 풍부한 정보를 포함하는 학습 가능한 임베딩을 생성하는 것이고, 둘째는 비디오와 텍스트 모달리티 간의 상호작용을 최적화하는 것입니다.\n\nCM2 프레임워크에서는 시각적 cross-attention 레이어와 텍스트 cross-attention 레이어를 분리하여 각 모달리티가 시간적 및 의미적 정보 처리를 독립적으로 수행할 수 있도록 합니다. 이는 Figure 2(c)에 설명되어 있으며, 비주얼 cross-attention에서는 인코딩된 시각적 특징과 이벤트 쿼리 간의 cross-attention을 고려하여 시간적 정보를 향상시키는 데 중점을 둡니다.\n\n이러한 구조는 각 모달리티가 자신의 정보를 보존하면서도 효과적으로 상호작용할 수 있도록 하여, 이벤트 로컬라이제이션과 캡셔닝의 성능을 개선하는 데 기여합니다. \n\n이와 관련된 내용은 논문의 3.2절에서 자세히 설명되어 있습니다.'

In [14]:
print(result)

해당 논문의 Versatile Encoder-Decoder는 비디오와 텍스트 모달리티를 효과적으로 처리하기 위해 설계된 구조입니다. 이 구조는 두 가지 주요 기능을 수행하는데, 첫째는 이벤트 쿼리(event queries)를 통해 시간적 및 의미적으로 풍부한 정보를 포함하는 학습 가능한 임베딩을 생성하는 것이고, 둘째는 비디오와 텍스트 모달리티 간의 상호작용을 최적화하는 것입니다.

CM2 프레임워크에서는 시각적 cross-attention 레이어와 텍스트 cross-attention 레이어를 분리하여 각 모달리티가 시간적 및 의미적 정보 처리를 독립적으로 수행할 수 있도록 합니다. 이는 Figure 2(c)에 설명되어 있으며, 비주얼 cross-attention에서는 인코딩된 시각적 특징과 이벤트 쿼리 간의 cross-attention을 고려하여 시간적 정보를 향상시키는 데 중점을 둡니다.

이러한 구조는 각 모달리티가 자신의 정보를 보존하면서도 효과적으로 상호작용할 수 있도록 하여, 이벤트 로컬라이제이션과 캡셔닝의 성능을 개선하는 데 기여합니다. 

이와 관련된 내용은 논문의 3.2절에서 자세히 설명되어 있습니다.


In [15]:
query = "해당 논문의 핵심 용어를 알려주세요."

result = chain.answer_generation(query)

print(result)

해당 논문의 핵심 용어는 다음과 같습니다:

1. **Dense Video Captioning**: 비디오 내의 이벤트를 밀집하게 캡션하는 기술로, 비디오의 모든 중요한 순간을 설명하는 것을 목표로 합니다.
2. **Cross-Modal Memory Retrieval**: 서로 다른 모달리티(예: 비디오와 텍스트) 간의 메모리 검색을 통해 정보를 효과적으로 활용하는 방법입니다.
3. **Encoder-Decoder Architecture**: 입력 데이터를 인코딩하고, 이를 기반으로 출력을 생성하는 구조로, 비디오 캡션 생성에 사용됩니다.
4. **Memory Construction**: 외부 메모리 뱅크를 구축하여 훈련 데이터의 문장 수준 특징을 활용하는 과정입니다.
5. **Feature Aggregation**: 여러 개의 문장 특징을 평균 풀링하여 유용한 정보를 요약하는 과정입니다.
6. **Localization Head**: 이벤트 세그먼트의 시작 시간, 종료 시간 및 신뢰도를 예측하는 모델의 구성 요소입니다.
7. **Captioning Head**: 캡션 생성을 위한 변형 가능한 소프트 어텐션 LSTM을 사용하는 모델의 구성 요소입니다.
8. **Training Losses**: 모델 훈련을 위한 손실 함수로, Lloc, Lcls, Lcount, Lcap 등이 포함됩니다.

이 용어들은 논문의 1장(Introduction)과 3장(Method)에서 주로 다루어집니다.
